In [ ]:
import urllib.request
import config

client_id = "1Jjc0fiWhpS6Rc_7IJyC" 
client_secret = "6cl1xQELBG" 

def makeURL(sNode, search_text ):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % sNode
    parameters = "?query=%s" % urllib.parse.quote(search_text)
    url = base + node + parameters
    return url


def requestURL(url):  # 매개인자로 url(http://www.naver.com.... ) 이 오면   open api 키를 header에 추가한 후, 크롤링
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)  # open api 키를 header에 추가
    req.add_header("X-Naver-Client-Secret", client_secret)  # open api 키를 header에 추가
    try:
        response = urllib.request.urlopen(req)  # 크롤링
        if response.status == 200:  # 200이면 정상 응답
            print("Url Request Success")
            data = response.read().decode('utf-8')
            return data  # 크롤링 결과 반환
    except Exception as e:  # url로 요청 중 오류가 발생시 실행
        print(e)
        print("Error for URL : %s" %url)
        return None



import json

def main():
    print("네이버 검색 프로그램입니다.")
    search_text = input(">검색할 키워드를 입력하세요.")
    #search_text = '탄핵'

    answer = input (">검색 카테고리를 선택하세요 \n 1. 네이버 뉴스,\
                    2. 네이버 블로그 3. 네이버 카페")
    sNode = 'news'
    if answer=='1' :
        sNode = 'news'  # 'news', 'blog', 'cafearticle'
    elif answer=='2' :
        sNode = 'blog'
    elif answer=='3' :
        sNode = 'cafearticle'
    print("*******",answer)
    targetURL = makeURL(sNode, search_text )    #1. URL 만들기
    result = requestURL(targetURL)              #2. URL로 크롤링
    #print(result)                              #3. 크롤링 결과 출력


    #4. 크롤링 전체 결과를 파일에 저장
    filename = '%s_naver_%s_all.json' % (search_text, sNode)
    with open(filename, 'w', encoding='utf8') as outfile:
        outfile.write(result)
    print(filename +' 저장 완료' )


    #5. 크롤링 결과 중 일부만 파싱할 때
    jsonResult = []
    if result != None:
        jsonSearch = json.loads(result)    # JSON 문자열을  파이썬 딕셔너리로 load

        for post in jsonSearch['items']:  # items에 있는 데이터만
            title = post['title']
            description = post['description']
            if sNode=='news' :  #news만 originallink가 있음
                org_link = post['originallink']
            else :
                org_link = ""
            #link = post['link']

            jsonResult.append({'title': title,
                               'description': description,
                               'org_link': org_link
                                # ,'link': link
                              })

    retJson = json.dumps(jsonResult,   #파이썬 딕셔너리를 json 문자열로
                         indent=4,     #들여쓰기 함
                         ensure_ascii=False) #한글 인코딩
    print(retJson)


    # 6. 파싱 결과 일부만 파일에 저장
    filename = '%s_naver_%s.json' % (search_text, sNode)
    with open(filename, 'w', encoding='utf8') as outfile:
        outfile.write(retJson)
    print(filename + ' 저장 완료')

main()

네이버 검색 프로그램입니다.
